# Project 10 — Interactive Implied Volatility Surface (Market Data or Synthetic) + SVI Calibration

## What this notebook does
- Builds an **implied volatility dataset** by maturity and strike:
  - **Option chain from yfinance** (if available), OR
  - a **synthetic surface** (fallback so the notebook always runs)
- Computes **implied vols** by inverting **Black–Scholes** on call mid-prices.
- Calibrates **SVI (Stochastic Volatility Inspired)** per maturity:
  \[
  w(k) = a + b\left(\rho (k-m) + \sqrt{(k-m)^2+\sigma^2}\right),
  \quad w=\sigma_{imp}^2 T,\quad k=\ln(K/F)
  \]
- Shows an **interactive 3D vol surface** (Plotly) + maturity slices (smiles)
- Runs basic **static arbitrage checks** (butterfly convexity + calendar monotonicity)

> Everything is written in **English**. Charts are **interactive** (Plotly).  
> Exported HTML charts are saved to the `assets/` folder.

## 0) Setup (paths + reproducibility)

In [1]:
from __future__ import annotations
from pathlib import Path
import numpy as np

SEED = 42
rng = np.random.default_rng(SEED)

PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())

CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects\10_vol_surface_svi
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\10_vol_surface_svi\assets


## 1) Imports

In [2]:
import math
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display, clear_output

from scipy.optimize import minimize, brentq
from scipy.stats import norm

# Optional market data
try:
    import yfinance as yf
    HAVE_YFINANCE = True
except Exception:
    HAVE_YFINANCE = False

HAVE_YFINANCE

True

## 2) Black–Scholes (Forward form) + Implied Vol solver

We price a European call using the forward \(F\) and discount factor \(DF=e^{-rT}\):
\[
C = DF \left(F N(d_1) - K N(d_2)\right), \quad
d_1 = \frac{\ln(F/K)+\tfrac{1}{2}\sigma^2T}{\sigma\sqrt{T}}, \quad d_2=d_1-\sigma\sqrt{T}.
\]

Implied volatility \(\sigma_{imp}\) is obtained by solving \(BS(\sigma)=C_{mkt}\) with a robust bracketing routine.

In [3]:
def bs_call_forward(F: float, K: float, T: float, r: float, sigma: float) -> float:
    """Black–Scholes call price using forward F. Returns price in spot currency."""
    if T <= 0:
        return max(0.0, (F - K))  # DF=1 at T=0 in forward measure; used only as a guard
    DF = math.exp(-r * T)
    if sigma <= 0:
        return DF * max(0.0, F - K)
    vol_sqrt = sigma * math.sqrt(T)
    d1 = (math.log(F / K) + 0.5 * sigma * sigma * T) / vol_sqrt
    d2 = d1 - vol_sqrt
    return DF * (F * norm.cdf(d1) - K * norm.cdf(d2))

def implied_vol_call(price: float, F: float, K: float, T: float, r: float, lo: float = 1e-8, hi: float = 5.0) -> float:
    """
    Robust implied vol for a call. Returns np.nan if price is outside no-arbitrage bounds
    or if a bracket cannot be found.
    """
    if T <= 0 or F <= 0 or K <= 0:
        return float("nan")

    DF = math.exp(-r * T)
    intrinsic = DF * max(0.0, F - K)
    upper = DF * F  # call upper bound

    if not (intrinsic - 1e-12 <= price <= upper + 1e-12):
        return float("nan")

    def f(sig: float) -> float:
        return bs_call_forward(F, K, T, r, sig) - price

    a, b = lo, hi
    fa, fb = f(a), f(b)

    # expand bracket if needed
    if fa * fb > 0:
        for b_try in [7.5, 10.0, 15.0, 20.0]:
            b = b_try
            fb = f(b)
            if fa * fb <= 0:
                break

    if fa * fb > 0:
        return float("nan")

    try:
        return float(brentq(f, a, b, maxiter=200))
    except Exception:
        return float("nan")

## 3) Getting market option data (yfinance) — with a clean fallback

If yfinance works, we:
- download the **call option chain** for several expiries
- build mid prices (bid/ask if available, else last price)
- compute \(T\) (year fraction) from expiry
- compute forward \(F \approx S_0 e^{rT}\) (dividend yield ignored by default for simplicity)

If download fails or data is empty, we generate a **synthetic implied vol surface**.

In [4]:
def _to_naive_utc(ts) -> pd.Timestamp:
    """
    Convert any datetime-like input to a timezone-naive UTC Timestamp.
    This prevents errors like: "Cannot subtract tz-naive and tz-aware datetime-like objects".
    """
    ts = pd.Timestamp(ts)
    if ts.tz is not None:
        ts = ts.tz_convert("UTC").tz_localize(None)
    return ts

def _today_utc() -> pd.Timestamp:
    # Always return tz-naive UTC
    return _to_naive_utc(pd.Timestamp.now(tz="UTC")).normalize()

def _yearfrac_act365(start: pd.Timestamp, end: pd.Timestamp) -> float:
    start = _to_naive_utc(start)
    end = _to_naive_utc(end)
    dt = (end - start).total_seconds()
    return max(0.0, dt / (365.0 * 24.0 * 3600.0))

def fetch_option_chain_yf(ticker: str, r: float = 0.02, max_expiries: int = 6) -> tuple[float, pd.DataFrame]:
    """
    Returns (S0, df_options) with columns:
    expiry, T, K, mid, iv, k=log(K/F), F, DF, volume, openInterest, bid, ask
    """
    if not HAVE_YFINANCE:
        raise RuntimeError("yfinance not installed")

    tk = yf.Ticker(ticker)

    # underlying spot
    hist = tk.history(period="10d", auto_adjust=True)
    if hist.empty:
        raise RuntimeError("Empty price history")
    S0 = float(hist["Close"].iloc[-1])

    expiries = list(tk.options)[:max_expiries]
    if len(expiries) == 0:
        raise RuntimeError("No option expiries returned")

    rows = []
    today = _today_utc()

    for exp in expiries:
        exp_dt = _to_naive_utc(pd.to_datetime(exp))
        T = _yearfrac_act365(today, exp_dt)
        if T <= 1e-6:
            continue

        chain = tk.option_chain(exp)
        calls = chain.calls.copy()
        if calls.empty:
            continue

        # mid price: prefer bid/ask mid when valid; fallback to lastPrice
        bid = calls.get("bid", pd.Series([np.nan]*len(calls)))
        ask = calls.get("ask", pd.Series([np.nan]*len(calls)))
        last = calls.get("lastPrice", pd.Series([np.nan]*len(calls)))

        mid_ba = (bid + ask) / 2.0
        mid = mid_ba.where((bid > 0) & (ask > 0) & (ask >= bid), last)

        # risk-free approximation + forward (dividend yield ignored)
        DF = math.exp(-r*T)
        F = S0 * math.exp(r*T)

        for i in range(len(calls)):
            K = float(calls["strike"].iloc[i])
            C = float(mid.iloc[i]) if pd.notna(mid.iloc[i]) else np.nan
            if not np.isfinite(C) or C <= 0:
                continue

            iv = implied_vol_call(C, F, K, T, r)
            if not np.isfinite(iv) or iv <= 0:
                continue

            k = math.log(K / F)

            rows.append({
                "ticker": ticker,
                "expiry": exp_dt.date().isoformat(),
                "T": float(T),
                "K": float(K),
                "mid": float(C),
                "iv": float(iv),
                "k": float(k),
                "F": float(F),
                "DF": float(DF),
                "volume": float(calls.get("volume", pd.Series([np.nan]*len(calls))).iloc[i]) if "volume" in calls else np.nan,
                "openInterest": float(calls.get("openInterest", pd.Series([np.nan]*len(calls))).iloc[i]) if "openInterest" in calls else np.nan,
                "bid": float(bid.iloc[i]) if pd.notna(bid.iloc[i]) else np.nan,
                "ask": float(ask.iloc[i]) if pd.notna(ask.iloc[i]) else np.nan,
            })

    df = pd.DataFrame(rows)
    if df.empty:
        raise RuntimeError("Option chain fetched but nothing usable after cleaning")

    # final cleaning
    df = df[df["iv"].between(0.01, 5.0)]
    df = df[df["K"] > 0].copy()
    return S0, df

def synthetic_surface(S0: float = 100.0, r: float = 0.02, seed: int = 42) -> tuple[float, pd.DataFrame]:
    """
    Synthetic implied vol surface with skew + smile + term structure, plus small noise.
    Always returns a non-empty surface.
    """
    rng_ = np.random.default_rng(seed)
    today = _today_utc()
    Ts = np.array([0.08, 0.16, 0.33, 0.5, 1.0, 2.0])  # in years
    strikes = np.linspace(0.6*S0, 1.4*S0, 31)

    rows = []
    for T in Ts:
        DF = math.exp(-r*T)
        F = S0 * math.exp(r*T)
        for K in strikes:
            k = math.log(K / F)
            # toy but realistic term structure + skew + smile
            base = 0.18 + 0.05*np.exp(-T) + 0.02*np.sqrt(T)
            skew = -0.22 * (1.0 / (1.0 + 2.0*T))
            smile = 0.35
            iv = base + skew*k + smile*(k**2)
            iv = max(0.03, iv + 0.005*rng_.standard_normal())

            C = bs_call_forward(F, K, T, r, iv)
            iv_back = implied_vol_call(C, F, K, T, r)

            rows.append({
                "ticker": "SYNTH",
                "expiry": (today + pd.Timedelta(days=int(round(365*T)))).date().isoformat(),
                "T": float(T),
                "K": float(K),
                "mid": float(C),
                "iv": float(iv_back) if np.isfinite(iv_back) else float("nan"),
                "k": float(k),
                "F": float(F),
                "DF": float(DF),
                "volume": np.nan,
                "openInterest": np.nan,
                "bid": np.nan,
                "ask": np.nan,
            })
    df = pd.DataFrame(rows).dropna(subset=["iv"])
    return S0, df

## 4) SVI calibration per maturity (least squares on total variance)

Define total variance:
\[
w = \sigma_{imp}^2 T
\]

We fit SVI parameters \((a,b,\rho,m,\sigma)\) by minimizing:
\[
\min \sum_i \left(w(k_i) - w_i\right)^2
\]

We enforce constraints with a parameter transform:
- \(b = e^{b_\*}\) (positive)
- \(\sigma = e^{\sigma_\*}\) (positive)
- \(\rho = \tanh(\rho_\*)\) (in \((-1,1)\))

In [5]:
def svi_total_variance(k: np.ndarray, a: float, b: float, rho: float, m: float, sig: float) -> np.ndarray:
    return a + b * (rho*(k - m) + np.sqrt((k - m)**2 + sig**2))

def _unpack_svi(x: np.ndarray) -> tuple[float, float, float, float, float]:
    a = x[0]
    b = math.exp(x[1])
    rho = math.tanh(x[2])
    m = x[3]
    sig = math.exp(x[4])
    return a, b, rho, m, sig

def fit_svi_one_maturity(k: np.ndarray, iv: np.ndarray, T: float, n_starts: int = 8, seed: int = 42) -> dict:
    rng_ = np.random.default_rng(seed)
    w_obs = (iv**2) * T

    # sensible defaults
    a0 = max(1e-8, float(np.nanmin(w_obs) * 0.8))
    # transformed parameters initial
    x0 = np.array([a0, math.log(0.1), np.arctanh(0.0), 0.0, math.log(0.2)])

    def obj(x: np.ndarray) -> float:
        a, b, rho, m, sig = _unpack_svi(x)
        w_fit = svi_total_variance(k, a, b, rho, m, sig)
        # penalties for negative total variance
        if np.any(w_fit <= 0):
            return 1e6 + float(np.sum((np.minimum(w_fit, 0))**2))
        return float(np.mean((w_fit - w_obs)**2))

    best = None
    best_val = float("inf")

    # multi-start for robustness
    for s in range(n_starts):
        if s == 0:
            x_init = x0.copy()
        else:
            x_init = x0.copy()
            x_init[0] = max(1e-8, a0 * (0.5 + 1.5*rng_.random()))
            x_init[1] = math.log(0.05 + 0.5*rng_.random())      # b
            x_init[2] = np.clip((rng_.random()*2 - 1)*0.8, -0.95, 0.95)
            x_init[2] = np.arctanh(x_init[2])                  # rho*
            x_init[3] = (rng_.random()*2 - 1) * 0.4            # m
            x_init[4] = math.log(0.05 + 0.5*rng_.random())     # sig

        res = minimize(obj, x_init, method="L-BFGS-B")
        val = float(res.fun)
        if val < best_val and res.success:
            best_val = val
            best = res.x.copy()

    if best is None:
        # fallback: accept best even if not success
        res = minimize(obj, x0, method="L-BFGS-B")
        best = res.x.copy()
        best_val = float(res.fun)

    a, b, rho, m, sig = _unpack_svi(best)
    return {"a": a, "b": b, "rho": rho, "m": m, "sig": sig, "mse": best_val, "T": float(T)}

def calibrate_svi_surface(df: pd.DataFrame, min_points: int = 10) -> pd.DataFrame:
    out = []
    for T, g in df.groupby("T"):
        g = g.dropna(subset=["iv", "k"])
        if len(g) < min_points:
            continue
        k = g["k"].values.astype(float)
        iv = g["iv"].values.astype(float)
        # remove absurd points
        mask = np.isfinite(k) & np.isfinite(iv) & (iv > 0.01) & (iv < 5.0)
        k = k[mask]; iv = iv[mask]
        if len(k) < min_points:
            continue
        fit = fit_svi_one_maturity(k, iv, float(T), n_starts=10, seed=SEED)
        fit["n_points"] = int(len(k))
        out.append(fit)
    return pd.DataFrame(out).sort_values("T")

def svi_iv_from_params(k: np.ndarray, T: float, params: dict) -> np.ndarray:
    w = svi_total_variance(k, params["a"], params["b"], params["rho"], params["m"], params["sig"])
    w = np.maximum(w, 1e-12)
    return np.sqrt(w / T)

## 5) Build surface objects + interactive plots

We will display:
- **3D scatter** of market implied vols (T, k, iv)
- **SVI surface** (mesh) on a uniform k-grid for each maturity
- **Smile slice**: market points + fitted curve at a selected maturity
- **Residual plot**: (market iv − model iv)

How to interpret:
- If the SVI mesh tracks market points well → good calibration
- Residual structure indicates systematic misfit (e.g., wings not captured)

In [6]:
def build_surface_grids(df: pd.DataFrame, svi_params: pd.DataFrame, k_min: float = None, k_max: float = None, n_k: int = 61):
    Ts = np.array(sorted(svi_params["T"].unique()))
    if k_min is None: k_min = float(df["k"].quantile(0.02))
    if k_max is None: k_max = float(df["k"].quantile(0.98))
    k_grid = np.linspace(k_min, k_max, n_k)

    Z = np.zeros((len(Ts), len(k_grid)))
    for i, T in enumerate(Ts):
        p = svi_params.loc[svi_params["T"] == T].iloc[0].to_dict()
        Z[i, :] = svi_iv_from_params(k_grid, float(T), p)

    return Ts, k_grid, Z

def plot_surface_market_and_svi(df: pd.DataFrame, Ts: np.ndarray, k_grid: np.ndarray, Z: np.ndarray, title: str):
    fig = go.Figure()

    # Market scatter
    fig.add_trace(go.Scatter3d(
        x=df["T"], y=df["k"], z=df["iv"],
        mode="markers", name="Market IV",
        marker=dict(size=3)
    ))

    # SVI surface
    # plotly Surface expects 2D arrays for z; x,y can be 1D
    fig.add_trace(go.Surface(
        x=Ts, y=k_grid, z=Z.T,
        name="SVI surface", showscale=False, opacity=0.65
    ))

    fig.update_layout(
        template="plotly_dark",
        title=title,
        scene=dict(
            xaxis_title="T (years)",
            yaxis_title="log-moneyness k=ln(K/F)",
            zaxis_title="Implied vol"
        ),
        height=700
    )
    return fig

def plot_smile_slice(df: pd.DataFrame, svi_params: pd.DataFrame, T_target: float, k_grid: np.ndarray):
    # choose nearest maturity in params
    Ts = svi_params["T"].values
    T_sel = float(Ts[np.argmin(np.abs(Ts - T_target))])
    g = df[df["T"] == T_sel].copy()

    p = svi_params.loc[svi_params["T"] == T_sel].iloc[0].to_dict()
    iv_fit = svi_iv_from_params(k_grid, T_sel, p)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=g["k"], y=g["iv"], mode="markers", name="Market", marker=dict(size=7)))
    fig.add_trace(go.Scatter(x=k_grid, y=iv_fit, mode="lines", name="SVI fit"))
    fig.update_layout(template="plotly_dark", title=f"Smile slice — T={T_sel:.3f}y", xaxis_title="k=ln(K/F)", yaxis_title="Implied vol")
    return fig, T_sel

def compute_residuals(df: pd.DataFrame, svi_params: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["iv_fit"] = np.nan
    for T in svi_params["T"].unique():
        p = svi_params.loc[svi_params["T"] == T].iloc[0].to_dict()
        idx = out["T"] == T
        out.loc[idx, "iv_fit"] = svi_iv_from_params(out.loc[idx, "k"].values, float(T), p)
    out["iv_resid"] = out["iv"] - out["iv_fit"]
    return out

## 6) Static arbitrage checks (basic)

This is a *pedagogical* check (not a full arbitrage-proof certification).

### Butterfly convexity (per maturity)
For a fixed maturity, call price as a function of strike must be **convex**:
\[
\frac{\partial^2 C}{\partial K^2} \ge 0
\]
We test convexity numerically on a strike grid built from \(K = F e^k\).

### Calendar monotonicity
Total variance \(w(k,T)\) should generally not decrease with maturity (weak condition):
\[
w(k,T_2) \ge w(k,T_1) \quad \text{for } T_2>T_1
\]

In [7]:
def butterfly_convexity_violations(T: float, F: float, r: float, k_grid: np.ndarray, iv_curve: np.ndarray, tol: float = 1e-10) -> float:
    """
    Returns fraction of negative second finite-difference (convexity violations) for call prices vs strike.
    """
    K = F * np.exp(k_grid)
    C = np.array([bs_call_forward(F, float(Ki), float(T), r, float(sig)) for Ki, sig in zip(K, iv_curve)])
    # second derivative approximation in strike space
    # assume uniform spacing in K
    dK = np.diff(K)
    if np.any(dK <= 0) or len(K) < 5:
        return float("nan")
    # simple second difference with non-uniform K: use a crude uniform approximation on K grid (close enough for diagnostics)
    C2 = C[2:] - 2*C[1:-1] + C[:-2]
    viol = np.mean(C2 < -tol)
    return float(viol)

def calendar_monotonicity_violations(Ts: np.ndarray, Z: np.ndarray, tol: float = 1e-12) -> float:
    """
    Z shape (nT, nk). total variance w = iv^2*T. Check w is non-decreasing in T.
    Returns fraction of (T,k) where w decreases.
    """
    w = (Z**2) * Ts[:, None]
    dw = np.diff(w, axis=0)
    viol = np.mean(dw < -tol)
    return float(viol)

## 7) One-click interactive dashboard

Controls:
- **Ticker** (e.g., SPY, AAPL, MSFT)
- **Use synthetic** (if market data fails)
- **Risk-free rate r**
- **Max expiries** to fetch
- **Filters** on OI/volume and bid/ask quality
- **Maturity selector** for smile slice

Outputs:
- Market scatter + SVI surface (3D)
- Smile slice + residual plot
- Arbitrage diagnostics

In [8]:
ticker_w = widgets.Text(value="SPY", description="Ticker")
use_synth_w = widgets.Checkbox(value=False, description="Use synthetic")
r_w = widgets.FloatSlider(value=0.02, min=0.0, max=0.08, step=0.0025, description="r")
max_exp_w = widgets.IntSlider(value=6, min=2, max=12, step=1, description="Max expiries")

min_oi_w = widgets.IntSlider(value=0, min=0, max=5000, step=50, description="Min OI")
min_vol_w = widgets.IntSlider(value=0, min=0, max=5000, step=50, description="Min Vol")
require_bidask_w = widgets.Checkbox(value=True, description="Prefer bid/ask mid (strict)")

build_btn = widgets.Button(description="Build surface", button_style="success")
out = widgets.Output()

display(widgets.VBox([
    widgets.HBox([ticker_w, use_synth_w, r_w, max_exp_w]),
    widgets.HBox([min_oi_w, min_vol_w, require_bidask_w, build_btn]),
    out
]))

def _apply_filters(df: pd.DataFrame, strict_bidask: bool) -> pd.DataFrame:
    df2 = df.copy()

    # basic iv sanity
    df2 = df2[np.isfinite(df2["iv"]) & (df2["iv"] > 0.01) & (df2["iv"] < 5.0)]

    # liquidity filters (optional)
    if "openInterest" in df2.columns:
        df2 = df2[(df2["openInterest"].fillna(0) >= min_oi_w.value)]
    if "volume" in df2.columns:
        df2 = df2[(df2["volume"].fillna(0) >= min_vol_w.value)]

    # strict bid/ask filter: keep only where bid/ask are valid
    # Note: many Yahoo chains have bid/ask = 0 even when lastPrice exists; strict mode can empty the dataset.
    if strict_bidask:
        if ("bid" in df2.columns) and ("ask" in df2.columns):
            ba_ok = (df2["bid"].fillna(-1) > 0) & (df2["ask"].fillna(-1) > 0) & (df2["ask"] >= df2["bid"])
            df2 = df2[ba_ok]

    return df2

def build_surface(_):
    with out:
        clear_output(wait=True)
        ticker = ticker_w.value.strip().upper()
        rr = float(r_w.value)
        max_exp = int(max_exp_w.value)

        # Load data
        if use_synth_w.value:
            S0, df = synthetic_surface(S0=100.0, r=rr, seed=SEED)
        else:
            try:
                S0, df = fetch_option_chain_yf(ticker, r=rr, max_expiries=max_exp)
            except Exception as e:
                print("⚠️ Market data failed -> switching to synthetic surface.")
                print("Reason:", e)
                S0, df = synthetic_surface(S0=100.0, r=rr, seed=SEED)

        # Apply filters: strict first, then relax automatically if it empties
        df_strict = _apply_filters(df, strict_bidask=bool(require_bidask_w.value))
        if df_strict.empty and require_bidask_w.value:
            print("⚠️ Strict bid/ask filtering removed all rows. Relaxing filter (keeping lastPrice-based mids).")
            df_strict = _apply_filters(df, strict_bidask=False)

        df = df_strict
        if df.empty:
            raise RuntimeError("No data after filtering. Reduce filters or tick 'Use synthetic'.")

        # Keep only maturities with enough points
        counts = df.groupby("T").size().sort_values(ascending=False)
        good_T = counts[counts >= 12].index.values
        df = df[df["T"].isin(good_T)].copy()

        if df.empty:
            raise RuntimeError("Not enough points per maturity after cleaning. Reduce filters or use synthetic mode.")

        print(f"Spot S0 ≈ {S0:.4f}")
        print("Maturities kept:", sorted(df["T"].unique()))
        display(df.head())

        # Calibrate SVI per maturity
        svi_params = calibrate_svi_surface(df, min_points=12)
        if svi_params.empty:
            raise RuntimeError("SVI calibration produced empty params. Try synthetic mode or reduce filters.")
        display(svi_params)

        Ts, k_grid, Z = build_surface_grids(df, svi_params, n_k=71)

        # Surface plot
        fig = plot_surface_market_and_svi(df, Ts, k_grid, Z, title="Implied Vol Surface — Market scatter + SVI fitted mesh")
        fig.show()
        fig.write_html(ASSETS_DIR / "vol_surface_market_vs_svi.html")

        # Smile slice selector
        T_slider = widgets.FloatSlider(
            value=float(Ts[len(Ts)//2]),
            min=float(Ts.min()),
            max=float(Ts.max()),
            step=float((Ts.max()-Ts.min())/50) if len(Ts) > 1 else 0.01,
            description="T slice"
        )
        display(T_slider)

        def show_slice(change=None):
            fig_s, T_sel = plot_smile_slice(df, svi_params, float(T_slider.value), k_grid)
            fig_s.show()

            resid = compute_residuals(df, svi_params)
            fig_r = px.scatter(
                resid[resid["T"] == T_sel],
                x="k", y="iv_resid",
                template="plotly_dark",
                title=f"Residuals (market iv − SVI iv) — T={T_sel:.3f}y"
            )
            fig_r.update_layout(xaxis_title="k", yaxis_title="IV residual")
            fig_r.show()

        T_slider.observe(show_slice, names="value")
        show_slice()

        # Arbitrage checks (on fitted surface)
        conv = []
        for i, T in enumerate(Ts):
            F = float(df[df["T"] == T]["F"].iloc[0])
            viol = butterfly_convexity_violations(float(T), F, rr, k_grid, Z[i, :], tol=1e-10)
            conv.append({"T": float(T), "butterfly_violation_rate": viol})
        conv_df = pd.DataFrame(conv)
        cal_viol = calendar_monotonicity_violations(Ts, Z, tol=1e-12)

        print("\n--- Static arbitrage diagnostics (basic) ---")
        display(conv_df)
        print(f"Calendar monotonicity violation rate (total variance decreasing): {cal_viol:.2%}")
        print("\nSaved:", (ASSETS_DIR / 'vol_surface_market_vs_svi.html').resolve())

build_btn.on_click(build_surface)
build_surface(None)

## 8) What to say in interviews (short script)

- Built an implied volatility dataset from option chains (cleaning, mid prices, maturities).
- Implemented robust **implied vol inversion** (no-arbitrage bounds + bracketing).
- Calibrated **SVI** per maturity and produced an interactive 3D volatility surface.
- Added diagnostics (smile slices, residuals) and basic static arbitrage checks.
- Delivered a parameter playground dashboard for “live” exploration.